In [23]:
import pandas as pd
import numpy as np
import sklearn as sk 
from scipy.stats import pearsonr, spearmanr, kendalltau
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier , plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

In [24]:
df = pd.read_csv(r'C:\Users\Varshith\Desktop\homeworks\Emory_HW\ML\hw3\loan_default.csv')

In [25]:
# df['term'] = df['term'].apply(lambda x: 1 if x == ' 60 months' else 0)

# grade_mapping = {'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1 }
# df['grade'] = df['grade'].replace(grade_mapping)
# df['grade'] = df['grade'].astype(int)

# df['emp_length'] = df['emp_length'].str.replace('<', '')
# df['emp_length'] = df['emp_length'].str.replace(' years', '')
# df['emp_length'] = df['emp_length'].str.replace(' year', '')
# df['emp_length'] = df['emp_length'].str.replace('n/a', '')
# df['emp_length'] = df['emp_length'].str.replace(' ', '')
# df['emp_length'] = df['emp_length'].str.replace('+', '')
# df['emp_length'] = df['emp_length'].apply(lambda x: 10 if x == "" else x)
# df['emp_length'] = df['emp_length'].fillna(0)
# df['emp_length'] = df['emp_length'].astype(int)

# df['home_ownership'] = df['home_ownership'].astype(str)
# df['home_ownership'] = df['home_ownership'].apply(lambda x: 1 if x in ['OWN', 'MORTGAGE'] else 0)
# df['home_ownership'] = df['home_ownership'].astype(int)

# df['verification_status'] = df['verification_status'].apply(lambda x: 1 if x in ['Verified', 'Source Verified'] else 0)

# df['purpose'] = df['purpose'].astype(str)
# df['purpose'] = df['purpose'].replace({
#     'home_improvement': 1, 'other': 1, 'moving': 1, 'vacation': 1,
#     'major_purchase': 1, 'small_business': 1, 'car': 1, 'medical': 1,
#     'house': 1, 'renewable_energy': 1, 'wedding': 1
# })

# df['purpose'] = df['purpose'].apply(lambda x: 1 if x == 1 else 0)
# df['purpose'] = df['purpose'].astype(int)

# df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format='%d-%m-%Y').dt.year

In [26]:
le = LabelEncoder()
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()
le4 = LabelEncoder()
le5 = LabelEncoder()
le6 = LabelEncoder()
le.fit(df['term'])
df['term'] = le.transform(df['term'])
le1.fit(df['grade'])
df['grade'] = le1.transform(df['grade'])
le2.fit(df['emp_length'])
df['emp_length'] = le2.transform(df['emp_length'])
le3.fit(df['home_ownership'] )
df['home_ownership']  = le3.transform(df['home_ownership'] )
le4.fit(df['verification_status'])
df['verification_status'] = le4.transform(df['verification_status'])
le5.fit(df['purpose'])
df['purpose'] = le5.transform(df['purpose'])
le6.fit(df['earliest_cr_line'])
df['earliest_cr_line'] = le6.transform(df['earliest_cr_line'])


In [27]:
# df = df.drop(["earliest_cr_line"], axis = 1)
train, test = train_test_split(df, test_size=0.15, random_state=42) 
X_train_whole = train.drop(['class'], axis=1).values
Y_train_whole = train['class'].values
train, val = train_test_split(train, test_size=0.15, random_state=42)

In [28]:
X_train = train.drop(['class'], axis=1).values
y_train = train['class'].values
X_val = val.drop(['class'], axis=1).values
y_val = val['class'].values
X_test = test.drop(['class'], axis=1).values
y_test = test['class'].values

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train) 
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val) 
X_test = scaler.transform(X_test)

In [8]:
def rank_correlation(x, y):
    corrs = []
    for i in range(x.shape[1]):
        corr,_ = pearsonr(x[:,i], y)
        corrs.append(corr)
    rank = np.argsort(np.abs(corrs))[::-1]
    return rank

In [9]:
rank_correlation(X_train_whole,Y_train_whole)

array([20,  3,  5, 11,  2,  8, 18,  9, 14,  6, 25, 24,  0,  7, 19,  1, 12,
       17,  4, 10, 23, 22, 16, 13, 21, 15], dtype=int64)

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

def tune_nn(x, y, hidden_params, act_params, alpha_params):

    model = MLPClassifier()

    param_grid = {
    'hidden_layer_sizes': hidden_params,
    'activation': act_params, 
    'alpha': alpha_params
    }

    grid = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc')

    grid.fit(x, y)

    best_params = {
    'best-hidden': grid.best_params_['hidden_layer_sizes'],
    'best-activation': grid.best_params_['activation'],
    'best-alpha': grid.best_params_['alpha']
    }

    return best_params

In [30]:
hidden_params = [(10,), (50,), (50,20), (100,), (100, 50), (200, 100), (300, 200, 100)]
act_params = ['logistic', 'tanh', 'relu']
alpha_params = [0.0001, 0.001, 0.01, 0.1]
best_params = tune_nn(X_val, y_val, hidden_params, act_params, alpha_params)

print("Best hidden layers:", best_params['best-hidden']) 
print("Best activation:", best_params['best-activation'])
print("Best alpha:", best_params['best-alpha'])

Best hidden layers: (100, 50)
Best activation: relu
Best alpha: 0.001


In [31]:
best_params

{'best-hidden': (100, 50), 'best-activation': 'relu', 'best-alpha': 0.001}

In [33]:
from sklearn.metrics import roc_auc_score, f1_score, fbeta_score
import time
nn = MLPClassifier(hidden_layer_sizes=best_params['best-hidden'], activation=best_params['best-activation'],
                   alpha=best_params['best-alpha'])

start_time = time.time()

nn.fit(X_train_whole, Y_train_whole)

duration = time.time() - start_time

# y_pred = nn.predict_proba(X_test)[:,1]
y_pred = nn.predict(X_test)
auc = roc_auc_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred) 
f2 = fbeta_score(y_test, y_pred, beta=2)

print("AUC:", auc)
print("F1:", f1)
print("F2:", f2)
print(duration)

AUC: 0.6907603890362511
F1: 0.5521472392638037
F2: 0.4403131115459883
1.1850543022155762


In [34]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

def tune_dt(x, y, dparams, lsparams):

        dt = DecisionTreeClassifier()
        param_grid = {
            'max_depth': dparams,
            'min_samples_leaf': lsparams
        }

        grid_search = GridSearchCV(dt, param_grid, scoring='roc_auc', cv=5)
        grid_search.fit(x, y)
        auc_scores = grid_search.cv_results_['mean_test_score'].reshape(len(depths), len(min_leaf_samples))
        mean_test_auc = grid_search.cv_results_['mean_test_score']  # Mean AUC scores for each parameter combination
        params = grid_search.cv_results_['params']  # Parameters corresponding to each AUC score
        best_depth = grid_search.best_params_['max_depth']
        best_leaf_samples = grid_search.best_params_['min_samples_leaf']
        best_auc = grid_search.best_score_
        result_dict = {
            "best-depth": best_depth,
            "best-leaf-samples": best_leaf_samples,
            "best-auc": best_auc,
            "auc-scores": auc_scores
        }

        return result_dict

In [35]:
depths = [2, 3, 5, 8, 10]
min_leaf_samples = [3, 5, 10, 15, 20] 
result = tune_dt(X_val, y_val, depths, min_leaf_samples)
auc_values = result['auc-scores']

In [36]:
dt = DecisionTreeClassifier(max_depth=result['best-depth'], min_samples_leaf=result['best-leaf-samples'])

start_time = time.time()

dt.fit(X_train_whole, Y_train_whole) 

duration = time.time() - start_time

y_pred = dt.predict(X_test)
auc = roc_auc_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
f2 = fbeta_score(y_test, y_pred, beta=2)
print("AUC: ", auc, "F1: ", f1, "F2: ", f2)
print(duration)

AUC:  0.754420866489832 F1:  0.6666666666666667 F2:  0.5805243445692884
0.046002864837646484
